<a href="https://colab.research.google.com/github/esephaneli/oto_ekspertiz/blob/main/oto_ekspertiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚠️ Bu sistem kesin ekspertiz değildir.
Sadece görsel + kullanıcı beyanına dayalı bir ön değerlendirmedir.
Kritik güvenlik bileşenleri için profesyonel kontrol önerilir.


In [ ]:
# 1) Client (OpenAI SDK) -> Gemini endpoint (OpenAI-compatible)

load_dotenv()
client = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://gener...."
)
MODEL = "gemini-...."


# 2) Input standardization (master data yok → etiketle)

def tag_damage_type(ui_value: str) -> str:
    return {
        "Çarpma/Çarpışma": "impact_collision",
        "Sürtme": "scrape_minor",
        "Devrilme / Takla": "rollover",
        "Dolu": "hail",
        "Sel / Su": "flood_water",
        "Yangın": "fire",
        "Hırsızlık / Vandalizm": "vandal_theft",
    }.get(ui_value, "unknown")


# 3) Structured context builder (form → vehicle_context JSON)

def build_vehicle_context(form) -> dict:
    return {
        "vehicle_context": {
            "vehicle_type": form.vehicle_type,
            "make": form.make, "model": form.model, "model_year": form.year,
            "mileage_km": form.km,
            "vehicle_profile": {
                "is_plated": form.is_plated,
                "is_towed_documented": form.towed,
                "is_scrap_documented": form.scrap,
                "known_major_history": {
                    "flood_water": form.flood,
                    "fire": form.fire,
                    "heavy_damage_recorded": form.heavy_damage
                }
            },
            "incident": {
                "damage_type_ui": form.damage_type,
                "damage_type_tag": tag_damage_type(form.damage_type),
                "user_free_text": form.notes
            }
        }
    }


# 4) Multimodal packaging (images → data URL)

def images_to_data_urls(paths: list[str]) -> list[dict]:
    items = []
    for p in paths:
        b64 = base64.b64encode(open(p, "rb").read()).decode("utf-8")
        items.append({"type": "image_url",
                      "image_url": {"url": f"data:image/jpeg;base64,{b64}"}})
    return items


# 5) Model call (chat.completions) + JSON-only output

def run_pre_inspection(form, image_paths):
    ctx = build_vehicle_context(form)
    prompt = f"""
    vehicle_context JSON:
    {json.dumps(ctx, ensure_ascii=False)}

    Output: return ONLY valid JSON:
    summary, damage_regions(severity), risk_flags, recommended_checks,
    estimated_cost_range_try, confidence, disclaimer
    """

    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": [{"type":"text","text": prompt},
                                         *images_to_data_urls(image_paths)]},
        ],
        temperature=0.2
    )

    result_text = resp.choices[0].message.content
    return safe_json_loads(result_text)


# 6) UI Layer (Gradio) -> show summary/checklist/raw_json
# - Upload photos + enter vehicle info
# - Click "Analyze"
# - Render: Summary + Checklist + JSON report

“Çıktı JSON olduğu için raporlanabilir ve ileride PDF/CRM entegrasyonuna hazır.”

“Master data olmadan çalışması için hasar türlerini standart etiketlere map’ledim.”